In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Word2VecFeature import Word2VecFeatureGenerator
from sklearn.model_selection import train_test_split

from utils import NewsContent

from gensim.models.word2vec import LineSentence
from nltk import sent_tokenize
import numpy as np
import pandas as pd

torch.manual_seed(1)
torch.cuda.is_available()


True

In [3]:
data = NewsContent('../fakenewsnet_dataset', ['politifact'], ['fake', 'real'])
w2v = Word2VecFeatureGenerator()

/home/ljun/anaconda3/envs/fakenews/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


model trained


/home/ljun/anaconda3/envs/fakenews/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


model saved into w2c_model


In [4]:
vocab_size = len(w2v.model.wv.vocab)
print(vocab_size)
w2v_weight = torch.FloatTensor(w2v.model.wv.vectors)
embedding = nn.Embedding.from_pretrained(w2v_weight)
print(embedding)
EMBEDDING_DIM = 300
HIDDEN_DIM = 6

98348
Embedding(98348, 300)


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, data):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(data.view(len(data), 1, -1), hidden_dim)
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

In [ ]:
model = LSTM(vocab_size, HIDDEN_DIM, batch_size=6, num_layers=1)

In [ ]:
loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=0.2)

#####################
# Train model
#####################
num_epochs = 10
sample_train = pd.read_csv("w2v_feature.csv").sample(800)
labels = sample_train["label"]
sample_train = sample_train.drop('label', axis=1).values
X_train, X_test, y_train, y_test = train_test_split(sample_train, labels, test_size=0.2, random_state=1)
X_train.shape

In [ ]:
hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

In [20]:
tokenizer = torch.hub.load('huggingface/pytorch-pretrained-BERT', 'bertTokenizer', 'bert-base-uncased', do_basic_tokenize=True)
# with open("news_corpus.txt", 'r') as f:
#     text = f.read()
from nltk import word_tokenize, sent_tokenize
# text = list(data.get_features())
# word_tokens = tokenizer.tokenize(text)
# sent_tokens = sent_tokenize(text)
for news in data.get_features():
    word_tokens.append(tokenizer.tokenize(str(news)))
    sent_tokens.append(sent_tokenize(str(news)))
# tokenized_text = tokenizer.tokenize(text)
# index_token = tokenizer.convert_tokens_to_ids(tokenized_text)



Using cache found in /home/ljun/.cache/torch/hub/huggingface_pytorch-pretrained-BERT_master


AttributeError: 'list' object has no attribute 'shape'

In [21]:
# tokens_tensor = torch.tensor([index_token])
np.array(word_tokens[:10])
# sent_tokens[:10]


array([list(['[', "'", 'listen', "'", ',', "'", 'curious', "'", ',', "'", 'rob', '##oca', '##l', "'", ',', "'", 'seek', "'", ',', "'", 'dam', '##ag', "'", ',', "'", 'inform', "'", ',', "'", 'moore', '##list', '##en', "'", ',', "'", 'curious', "'", ',', "'", 'rob', '##oca', '##l', "'", ',', "'", 'seek', "'", ',', "'", 'dam', '##ag', "'", ',', "'", 'inform', "'", ',', "'", 'moor', "'", ',', "'", 'copyright', "'", ',', "'", 'w', '##kr', '##g', "'", ',', "'", 'right', "'", ',', "'", 'res', '##er', '##v', "'", ',', "'", 'video', "'", ',', "'", 'bill', "'", ',', "'", 'ri', '##al', "'", ',', "'", 'up', '##da', '##t', "'", ',', "'", 'w', '##kr', '##g', "'", ',', "'", 'ex', '##ec', '##ut', "'", ',', "'", 'editor', "'", ',', "'", 'mart', '##i', "'", ',', "'", 'baron', "'", ',', "'", 'washington', "'", ',', "'", 'post', "'", ',', "'", 're', '##lea', '##s', "'", ',', "'", 'follow', "'", ',', "'", 'statement', "'", ',', "'", 'post', "'", ',', "'", 'learn', "'", ',', "'", 'least', "'", ',', "'", 'on